# Json_parse.py
Read Json data and returna
- prices_df: pd.DataFrame
- esg_s: pd.Series

In [1]:
# add the parent directory to Python path
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

# load test json in jupyter notebook
import json
with open("./small_dataset.json") as f:
    dic = json.load(f)
    dic = dic["data"] # when run Launcher.ipynb, comment this out

# Parse data into assets and
from utils import json_parse
price_df, esg_s, asset_arr = json_parse(dic)

# Pre_processing.py
Uses `price_df` to obtain:
- μ
- Σ

In [2]:
# Calculates mu and sigma using OAS, LedoitWolf or normal cov
from utils import compute_mu_sigma
mu, Sigma = compute_mu_sigma(price_df, lookback=252, shrinkage=True)

/home/jovyan/work/git/singapore25_challenge3_team9/et/../utils/Pre_processing.py:28: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices_df.pct_change().dropna() # percentage change between the current element and prior element


# Classical Algos

Hello

In [3]:
from utils import run_portfolio_optimization_suite

config_example = {
    'compute_inputs': {'shrinkage': True},
    'scalarized_qp': {
        'lamb_grid': [0.5, 1.0, 2.0],
        'eta_grid': [0.0, 0.2, 0.5],
        'lb': 0.0, 'ub': 0.2, 'allow_short': False # Example: Max 20% per asset
    },
    'nsga2': {
        'pop_size': 150, 'generations': 80, 'plot': True
    },
    'cardinality': {
        'k': 15, 'lamb': 1.0, 'eta': 0.2
    },
    'cvar': {
        'alpha': 0.90, 'lamb': 0.5, 'eta': 0.3, 'use_mu_in_obj': True
    }
}

# Assuming prices_df and esg_df are loaded pandas DataFrames
results_dict = run_portfolio_optimization_suite(price_df, mu, Sigma, asset_arr, esg_s, config_example)
print("\n--- Results Summary ---")
print(results_dict.keys())

Inputs computed for 13 asset_arr.

Running Scalarized QP grid search...
  lambda=0.50, eta=0.00 -> ret=0.3695, vol=0.1291, esg=6.5180
  lambda=0.50, eta=0.20 -> ret=0.1788, vol=0.1084, esg=9.4253
  lambda=0.50, eta=0.50 -> ret=0.0680, vol=0.1179, esg=9.6040
  lambda=1.00, eta=0.00 -> ret=0.3695, vol=0.1291, esg=6.5180
  lambda=1.00, eta=0.20 -> ret=0.1788, vol=0.1084, esg=9.4254
  lambda=1.00, eta=0.50 -> ret=0.1788, vol=0.1084, esg=9.4253
  lambda=2.00, eta=0.00 -> ret=0.3695, vol=0.1291, esg=6.5180
  lambda=2.00, eta=0.20 -> ret=0.2725, vol=0.1159, esg=8.8167
  lambda=2.00, eta=0.50 -> ret=0.1788, vol=0.1084, esg=9.4253

Running CVaR Optimization...
  CVaR alpha=0.9, return lambda=0.50, ESG eta=0.30
  Non-zero weights: 5
  CVaR Metrics: ret=0.1882, vol=0.2036, esg=11.0000

Optimization suite finished.

--- Results Summary ---
dict_keys(['inputs', 'scalarized_qp_results', 'cvar_result'])
